# DQN

Метод обучения DQN — это нейросетевая адаптация алгоритма Q-learning. Также для него разработан набор дополнений, которые становятся актуальными при переходе к обучению глубоких нейронных сетей и решению более сложных задач (то есть задач с бОльшим пространством состояний).

Реализуем алгоритм DQN для решения среды [CartPole](https://gymnasium.farama.org/environments/classic_control/cart_pole/), цель которой балансировать палочкой в вертикальном положении, управляя только тележкой, к которой она прикреплена. Будем использовать библиотеку PyTorch для обучения нейронной сети, аппроксимирующей Q-функцию (но вы можете воспользоваться и любой другой библиотекой для обучения глубоких сетей, таких как TensorFlow или Jax).

![cartpole](https://gymnasium.farama.org/_images/cart_pole.gif)

![cartpole](https://www.researchgate.net/publication/362568623/figure/fig5/AS:1187029731807278@1660021350587/Screen-capture-of-the-OpenAI-Gym-CartPole-problem-with-annotations-showing-the-cart.png)

In [1]:
# Cтавим нужные зависимости, если это колаб
try:
    import google.colab
    COLAB = True
except ModuleNotFoundError:
    COLAB = False
    pass

if COLAB:
    !pip -q install "gymnasium[classic-control, atari, accept-rom-license]"
    !pip -q install piglet
    !pip -q install imageio_ffmpeg
    !pip -q install moviepy==1.0.3
    !pip -q install setuptools==59.8.0

In [2]:
import abc
import base64
import io
import math
import os
import random
import time
from dataclasses import dataclass

# import cv2
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
import pygame
from gymnasium import spaces
from gymnasium.envs.registration import WrapperSpec
%matplotlib inline

if COLAB:
    from google.colab import files
    from google.colab.patches import cv2_imshow
    from google.colab import output

### Action Space

The action is a `ndarray` with shape `(1,)` which can take values `{0, 1}` indicating the direction of the fixed force the cart is pushed with.

- 0: Push cart to the left
- 1: Push cart to the right

**Note**: The velocity that is reduced or increased by the applied force is not fixed and it depends on the angle
the pole is pointing. The center of gravity of the pole varies the amount of energy needed to move the cart underneath it

### Observation Space

The observation is a `ndarray` with shape `(4,)` with the values corresponding to the following positions and velocities:

| Num | Observation           | Min                 | Max               |
|-----|-----------------------|---------------------|-------------------|
| 0   | Cart Position         | -4.8                | 4.8               |
| 1   | Cart Velocity         | -Inf                | Inf               |
| 2   | Pole Angle            | ~ -0.418 rad (-24°) | ~ 0.418 rad (24°) |
| 3   | Pole Angular Velocity | -Inf                | Inf               |

**Note:** While the ranges above denote the possible values for observation space of each element, it is not reflective of the allowed values of the state space in an unterminated episode. Particularly:

- The cart x-position (index 0) can be take values between `(-4.8, 4.8)`, but the episode terminates if the cart leaves the `(-2.4, 2.4)` range.
- The pole angle can be observed between  `(-.418, .418)` radians (or **±24°**), but the episode terminates
   if the pole angle is not in the range `(-.2095, .2095)` (or **±12°**)

### Rewards

Since the goal is to keep the pole upright for as long as possible, a reward of `+1` for every step taken,
including the termination step, is allotted. The threshold for rewards is 500 for v1 and 200 for v0.

### Starting State

All observations are assigned a uniformly random value in `(-0.05, 0.05)`

### Episode End

The episode ends if any one of the following occurs:

1. Termination: Pole Angle is greater than ±12°
2. Termination: Cart Position is greater than ±2.4 (center of the cart reaches the edge of the display)
3. Truncation: Episode length is greater than 500 (200 for v0)

In [3]:
env = gym.make("CartPole-v1", max_episode_steps=1000)
env.reset()

# Выведем информацию о пространствах состояний и действий
print(f'{env.observation_space=}')
print(f'{env.action_space=}')

n_actions = env.action_space.n
state_dim = env.observation_space.shape
print(f'Action_space: {n_actions} | State_space: {state_dim}')

env.observation_space=Box([-4.8               -inf -0.41887903        -inf], [4.8               inf 0.41887903        inf], (4,), float32)
env.action_space=Discrete(2)
Action_space: 2 | State_space: (4,)


Т.к. описание состояния в задаче с маятником представляет собой не "сырые" признаки, а уже предобработанные (координаты, углы), нам не нужна для начала сложная архитектура, начнем с такой:
<img src="https://raw.githubusercontent.com/Tviskaron/mipt/master/2020/RL/figures/DQN.svg">

Будем использовать только полносвязные слои (``torch.nn.Linear``) и простые активационные функции (``torch.nn.ReLU``). Сигмоиды и другие похожие функции активации могут плохо работать с ненормализованными входными данными.

Будем приближать Q-функцию агента, минимизируя среднеквадратичную TD-ошибку:
$$
\delta = Q_{\theta}(s, a) - [r(s, a) + \gamma \cdot max_{a'} Q_{-}(s', a')]
$$
$$
L = \frac{1}{N} \sum_i \delta_i^2,
$$
где
* $s, a, r, s'$ состояние, действие, вознаграждение и следующее состояние
* $\gamma$ дисконтирующий множитель.

Основная тонкость состоит в использовании $Q_{-}(s',a')$. Это та же самая функция, что и $Q_{\theta}$, которая является выходом нейронной сети, но при обучении сети, мы не пропускаем через эти слои градиенты. В научных статьях можно обнаружить следующее обозначение для остановки градиента: $SG(\cdot)$. В PyTorch есть метод `.detach()` класса `Tensor`, который возвращает тензор с выключенными градиентами, а также контекстный менеджер `with torch.no_grad()`, который задает контекст с вычислениями, для которых не вычисляется градиент.

(1 балл)

In [ ]:
# !pip install torch

  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/109.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/109.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/109.3 MB 330.3 kB/s eta 0:05:31
   ---------------------------------------- 0.0/109.3 MB 326.8 kB/s eta 0:05:35
   ---------------------------------------- 0.1/109.3 MB 365.7 kB/s eta 0:04:59
   ---------------------------------------- 0.1/109.3 MB 403.5 kB/s eta 0:04:31
   ---------------------------------------- 0.1/109.3 MB 437.6 kB/s eta 0:04:10
   ---------------------------------------- 0.1/109.3 MB 448.2 kB/s eta 0:04:04
   ---------------------------------------- 0.2/109.3 MB 476.3 kB/s eta 0:03:50
   ---------------------------------------- 0.2/109.3 MB 498.9 kB/s eta 0:03:39
   ---------------------------


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\skfim\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [6]:
import torch
import torch.nn as nn

def create_network(input_dim, hidden_dims, output_dim):
    # network = nn.Sequential(
    #    torch.nn.Linear(input_dim, ...),
    #    torch.nn.ReLU() or Tanh(),
    #    ...
    # )
    ####### Здесь ваш код ########
    layers = []
    prev_dim = input_dim
    
    for hidden_dim in hidden_dims:
        layers.append(nn.Linear(prev_dim, hidden_dim))
        layers.append(nn.ReLU())
        prev_dim = hidden_dim
    
    layers.append(nn.Linear(prev_dim, output_dim))
    network = nn.Sequential(*layers)
    ##############################
    return network

Добавим $\epsilon$-жадный выбор действий (1 балл):

In [7]:
def select_action_eps_greedy(Q, state, epsilon):
    """Выбирает действие epsilon-жадно."""
    if not isinstance(state, torch.Tensor):
        state = torch.tensor(state, dtype=torch.float32)
    Q_s = Q(state).detach().numpy()

    # action =
    ####### Здесь ваш код ########
    if np.random.random() < epsilon:
        action = np.random.randint(0, len(Q_s))
    else:
        action = np.argmax(Q_s)
    ##############################

    action = int(action)
    return action


Q = create_network(
    input_dim=np.prod(state_dim), hidden_dims=[64, 64], output_dim=n_actions
)
select_action_eps_greedy(Q, env.reset()[0].flatten(), epsilon=0.1)

1

(4 балла)

In [8]:
def to_tensor(x, dtype=np.float32):
    if isinstance(x, torch.Tensor):
        return x
    x = np.asarray(x, dtype=dtype)
    x = torch.from_numpy(x)
    return x

def compute_td_target(
        Q, rewards, next_states, terminateds, gamma=0.99, check_shapes=True,
):
    """ Считает TD-target."""

    # переводим входные данные в тензоры
    r = to_tensor(rewards)  # shape: [batch_size]
    s_next = to_tensor(next_states)  # shape: [batch_size, state_size]
    term = to_tensor(terminateds, bool)  # shape: [batch_size]

    # получаем Q[s_next, .] — значения полезности всех действий в следующем состоянии
    # Q_sn = ...,
    # а затем вычисляем V^*[s_next] — оптимальные значения полезности следующем состоянии
    # V_sn = ...
    ####### Здесь ваш код ########
    with torch.no_grad():
        Q_sn = Q(s_next)  # shape: [batch_size, n_actions]
    V_sn = torch.max(Q_sn, dim=1)[0]  # shape: [batch_size] - максимум по оси действий
    ##############################

    assert V_sn.dtype == torch.float32

    # вычисляем TD target
    # target = ...
    ####### Здесь ваш код ########
    # Если terminated, то target = r (нет будущего)
    # Иначе target = r + gamma * V_sn
    target = r + gamma * V_sn * (~term).float()
    ##############################

    if check_shapes:
        assert Q_sn.data.dim() == 2, \
            "убедитесь, что вы предсказали q-значения для всех действий в следующем состоянии"
        assert V_sn.data.dim() == 1, \
            "убедитесь, что вы вычислили V (s ') как максимум только по оси действий, а не по всем осям"
        assert target.data.dim() == 1, \
            "что-то не так с целевыми q-значениями, они должны быть вектором"

    return target


def compute_td_loss(
        Q, states, actions, td_target, regularizer=.1, out_non_reduced_losses=False
):
    """ Считает TD ошибку."""

    # переводим входные данные в тензоры
    s = to_tensor(states)  # shape: [batch_size, state_size]
    a = to_tensor(actions, int).long()  # shape: [batch_size]

    # получаем Q[s, a] для выбранных действий в текущих состояниях
    # (для каждого примера из батча)
    # Q_s_a = ...
    ####### Здесь ваш код ########
    Q_s_all = Q(s)  # shape: [batch_size, n_actions]
    Q_s_a = Q_s_all[torch.arange(len(a)), a]  # shape: [batch_size] - выбираем Q[s, a] для каждого примера
    ##############################

    # вычисляем TD error
    # td_error = ...
    ####### Здесь ваш код ########
    td_error = Q_s_a - td_target
    ##############################

    # MSE loss для минимизации
    td_losses = td_error ** 2
    loss = torch.mean(td_losses)
    # добавляем L1 регуляризацию на значения Q
    loss += regularizer * torch.abs(Q_s_a).mean()

    if out_non_reduced_losses:
        return loss, td_losses.detach()

    return loss

In [9]:
def eval_dqn(env_name, Q):
    """Оценка качества работы алгоритма на одном эпизоде"""
    env = gym.make(env_name)
    s, _ = env.reset()
    done, ep_return = False, 0.

    while not done:
        # set epsilon = 0 to make an agent act greedy
        a = select_action_eps_greedy(Q, s, epsilon=0.)
        s_next, r, terminated, truncated, _ = env.step(a)
        done = terminated or truncated
        ep_return += r
        s = s_next

        if done:
            break

    return ep_return

In [10]:
from collections import deque

def linear(st, end, duration, t):
    """
    Линейная интерполяция значений в пределах диапазона [st, end],
    используя прогресс по времени t относительно всего отведенного
    времени duration.
    """

    if t >= duration:
        return end
    return st + (end - st) * (t / duration)

def run_dqn(
        env_name="CartPole-v1",
        hidden_dims=(128, 128), lr=1e-3, gamma=0.99,
        eps_st=.4, eps_end=.02, eps_dur=.25, total_max_steps=100_000,
        train_schedule=1, eval_schedule=1000, smooth_ret_window=10, success_ret=200.
):
    env = gym.make(env_name)
    eval_return_history = deque(maxlen=smooth_ret_window)

    Q = create_network(
        input_dim=env.observation_space.shape[0], hidden_dims=hidden_dims, output_dim=env.action_space.n
    )
    opt = torch.optim.Adam(Q.parameters(), lr=lr)

    s, _ = env.reset()
    done = False

    for global_step in range(1, total_max_steps + 1):
        epsilon = linear(eps_st, eps_end, eps_dur * total_max_steps, global_step)

        a = select_action_eps_greedy(Q, s, epsilon=epsilon)
        s_next, r, terminated, truncated, _ = env.step(a)
        done = terminated or truncated

        if global_step % train_schedule == 0:
            opt.zero_grad()
            td_target = compute_td_target(Q, [r], [s_next], [terminated], gamma=gamma)
            loss = compute_td_loss(Q, [s], [a], td_target)
            loss.backward()
            opt.step()

        if global_step % eval_schedule == 0:
            eval_return = eval_dqn(env_name, Q)
            eval_return_history.append(eval_return)
            avg_return = np.mean(eval_return_history)
            print(f'{global_step=} | {avg_return=:.3f} | {epsilon=:.3f}')
            if avg_return >= success_ret:
                print('Решено!')
                break

        s = s_next
        if done:
            s, _ = env.reset()
            done = False

run_dqn(eval_schedule=250)

global_step=250 | avg_return=10.000 | epsilon=0.396
global_step=500 | avg_return=13.500 | epsilon=0.392
global_step=750 | avg_return=13.000 | epsilon=0.389
global_step=1000 | avg_return=16.250 | epsilon=0.385
global_step=1250 | avg_return=17.400 | epsilon=0.381
global_step=1500 | avg_return=16.167 | epsilon=0.377
global_step=1750 | avg_return=17.143 | epsilon=0.373
global_step=2000 | avg_return=16.875 | epsilon=0.370
global_step=2250 | avg_return=17.778 | epsilon=0.366
global_step=2500 | avg_return=19.600 | epsilon=0.362
global_step=2750 | avg_return=30.900 | epsilon=0.358
global_step=3000 | avg_return=37.400 | epsilon=0.354
global_step=3250 | avg_return=42.200 | epsilon=0.351
global_step=3500 | avg_return=43.900 | epsilon=0.347
global_step=3750 | avg_return=57.000 | epsilon=0.343
global_step=4000 | avg_return=59.100 | epsilon=0.339
global_step=4250 | avg_return=58.000 | epsilon=0.335
global_step=4500 | avg_return=58.800 | epsilon=0.332
global_step=4750 | avg_return=69.700 | epsilon=0.

Комментарии к получаемым результатам:
- `avg_return` - это средняя отдача за эпизод на истории из последних десяти эпизодов. В случае корректной реализации, этот показатель будет низким первые 1000 шагов и только затем будет возрастать и сойдется на 5000-15000 шагах в зависимости от архитектуры сети.
- Если сеть не достигает нужных результатов к концу цикла, попробуйте увеличить число нейронов в скрытом слое или поменяйте начальный $\epsilon$.
- Переменная `epsilon` обеспечивает стремление агента исследовать среду. В данной реализации используется линейное затухание для частоты исследования.

### DQN with Experience Replay

Теперь попробуем добавить поддержку памяти прецедентов (Replay Buffer), которая будет из себя представлять очередь из наборов: $\{(s, a, r, s', 1_\text{terminated})\}$.

Тогда во время обучения каждый новый переход будет добавляться в память, а обучение будет целиком производиться на переходах, просэмплированных из памяти прецедентов.

(1 балл)

In [11]:
def sample_batch(replay_buffer, n_samples):
    # sample randomly `n_samples` samples from replay buffer
    # and split an array of samples into arrays:
    #    states, actions, rewards, next_states, terminateds
    # Use np.random.default_rng().choice for sampling
    ####### Здесь ваш код ########
    rng = np.random.default_rng()
    indices = rng.choice(len(replay_buffer), size=n_samples, replace=True)
    
    samples = [replay_buffer[i] for i in indices]
    states, actions, rewards, next_states, terminateds = zip(*samples)
    ##############################

    return np.array(states), np.array(actions), np.array(rewards), np.array(next_states), np.array(terminateds)

In [12]:
def run_dqn_rb(
        env_name="CartPole-v1",
        hidden_dims=(256, 256), lr=1e-3, gamma=0.99,
        eps_st=.4, eps_end=.02, eps_dur=.25, total_max_steps=100_000,
        train_schedule=4, replay_buffer_size=400, batch_size=32,
        eval_schedule=1000, smooth_ret_window=5, success_ret=200.
):
    env = gym.make(env_name)
    replay_buffer = deque(maxlen=replay_buffer_size)
    eval_return_history = deque(maxlen=smooth_ret_window)

    Q = create_network(
        input_dim=env.observation_space.shape[0], hidden_dims=hidden_dims, output_dim=env.action_space.n
    )
    opt = torch.optim.Adam(Q.parameters(), lr=lr)

    s, _ = env.reset()
    done = False

    for global_step in range(1, total_max_steps + 1):
        epsilon = linear(eps_st, eps_end, eps_dur * total_max_steps, global_step)
        a = select_action_eps_greedy(Q, s, epsilon=epsilon)
        s_next, r, terminated, truncated, _ = env.step(a)

        replay_buffer.append((s, a, r, s_next, terminated))
        done = terminated or truncated

        if global_step % train_schedule == 0:
            train_batch = sample_batch(replay_buffer, batch_size)
            states, actions, rewards, next_states, terminateds = train_batch

            opt.zero_grad()
            td_target = compute_td_target(Q, rewards, next_states, terminateds, gamma=gamma)
            loss = compute_td_loss(Q, states, actions, td_target)
            loss.backward()
            opt.step()

        if global_step % eval_schedule == 0:
            eval_return = eval_dqn(env_name, Q)
            eval_return_history.append(eval_return)
            avg_return = np.mean(eval_return_history)
            print(f'{global_step=} | {avg_return=:.3f} | {epsilon=:.3f}')
            if avg_return >= success_ret:
                print('Решено!')
                break

        s = s_next
        if done:
            s, _ = env.reset()
            done = False

run_dqn_rb(eval_schedule=250)

global_step=250 | avg_return=8.000 | epsilon=0.396
global_step=500 | avg_return=53.500 | epsilon=0.392
global_step=750 | avg_return=40.333 | epsilon=0.389
global_step=1000 | avg_return=43.000 | epsilon=0.385
global_step=1250 | avg_return=48.200 | epsilon=0.381
global_step=1500 | avg_return=51.600 | epsilon=0.377
global_step=1750 | avg_return=34.800 | epsilon=0.373
global_step=2000 | avg_return=45.000 | epsilon=0.370
global_step=2250 | avg_return=59.000 | epsilon=0.366
global_step=2500 | avg_return=76.200 | epsilon=0.362
global_step=2750 | avg_return=134.600 | epsilon=0.358
global_step=3000 | avg_return=161.000 | epsilon=0.354
global_step=3250 | avg_return=173.200 | epsilon=0.351
global_step=3500 | avg_return=192.200 | epsilon=0.347
global_step=3750 | avg_return=181.400 | epsilon=0.343
global_step=4000 | avg_return=149.800 | epsilon=0.339
global_step=4250 | avg_return=166.200 | epsilon=0.335
global_step=4500 | avg_return=212.000 | epsilon=0.332
Решено!


## DQN with Prioritized Experience Replay

Добавим каждому примеру, хранящемуся в памяти, значение приоритета. Приоритет будет влиять на частоту случайного выбора примеров в пакет на обучение. Удачный выбор приоритета позволит повысить эффективность обучения. Популярным вариантом является абсолютное значение TD-ошибки. Таким образом акцент при обучении Q-функции отводится примерам, на которых аппроксиматор ошибается сильнее.

Однако, нужно помнить, что это значение быстро устаревает, если его не обновлять. Но и обновлять для всей памяти каждый раз накладно. Из-за этого потребуется искать баланс между точностью оценки приоритета и скоростью работы.

В данном задании мы будем делать следующее:

- Использовать TD-ошибку в качестве приоритета.
- Так как для пакета данных, используемых при обучении, в любом случае будет вычислена TD-ошибка, воспользуемся полученными значениями для обновления значений приоритета в памяти для каждого примера из данного пакета.
- Будем периодически сортировать память для того, чтобы новые добавляемые переходы заменяли собой те переходы, у которых наименьший приоритет (т.е. наименьшие значения ошибки). Сортировка - дорогостоящая операция, поэтому выбрана редкая периодичность.

NB: Обратите внимание, что софтмакс очень чувствителен к масштабу величин и часто требует подбора температуры. Чтобы частично нивелировать эту проблему, предлагается использовать не `softmax(priorities)` напрямую, а воспользоваться функцией $\text{symlog} = \text{sign}(x) \cdot \log (|x| + 1)$, то есть `softmax(symlog(priorities))`, и не подбирать температуру. Идея взята из статьи DreamerV2 —-- в этой статье можно ознакомиться с идеей применения функций *symlog* и *simexp*, так как это полезная альтернатива нормализации некоторых величин в RL (вознаграждений, отдач, полезностей, логитов).

(2 балла)

In [15]:
def symlog(x):
    """
    Compute symlog values for a vector `x`.
    It's an inverse operation for symexp.
    """
    return np.sign(x) * np.log(np.abs(x) + 1)

def softmax(xs, temp=1.):
    exp_xs = np.exp((xs - xs.max()) / temp)
    return exp_xs / exp_xs.sum()

def sample_prioritized_batch(replay_buffer, n_samples):
    # Sample randomly `n_samples` examples from replay buffer
    # weighting by priority (example's TD error) and split an array
    # of sample tuples into arrays:
    #    states, actions, rewards, next_states, terminateds
    # Also, keep samples' indices (into `indices`) to return them too!
    # Note that each sample in replay buffer is a tuple:
    #   (priority, state, action, reward, next_state, terminated)
    # Use
    ####### Здесь ваш код ########
    if len(replay_buffer) == 0:
        raise ValueError("Replay buffer is empty")
    
    # Извлекаем приоритеты (первый элемент каждого tuple)
    priorities = np.array([sample[0] for sample in replay_buffer])
    
    # Добавляем небольшое значение к приоритетам, чтобы избежать нулевых вероятностей
    priorities = np.abs(priorities) + 1e-6
    
    # Применяем symlog для нормализации
    symlog_priorities = symlog(priorities)
    
    # Вычисляем softmax для получения вероятностей выборки
    probs = softmax(symlog_priorities, temp=1.0)
    
    # Выбираем индексы согласно вероятностям
    rng = np.random.default_rng()
    indices = rng.choice(len(replay_buffer), size=n_samples, p=probs, replace=True)
    
    # Извлекаем образцы по выбранным индексам
    samples = [replay_buffer[i] for i in indices]
    
    # Разделяем на массивы (пропускаем первый элемент - priority)
    # Каждый sample имеет структуру: (priority, state, action, reward, next_state, terminated)
    states = [s[1] for s in samples]
    actions = [s[2] for s in samples]
    rewards = [s[3] for s in samples]
    next_states = [s[4] for s in samples]
    terminateds = [s[5] for s in samples]
    ##############################

    batch = (
        np.array(states), np.array(actions), np.array(rewards),
        np.array(next_states), np.array(terminateds)
    )
    return batch, indices

def update_batch(replay_buffer, indices, batch, new_priority):
    """Updates batches with corresponding indices
    replacing their priority values."""
    states, actions, rewards, next_states, terminateds = batch

    for i in range(len(indices)):
        new_batch = (
            new_priority[i], states[i], actions[i], rewards[i],
            next_states[i], terminateds[i]
        )
        replay_buffer[indices[i]] = new_batch

def sort_replay_buffer(replay_buffer):
    """Sorts replay buffer to move samples with
    lesser priority to the beginning ==> they will be
    replaced with the new samples sooner."""
    new_rb = deque(maxlen=replay_buffer.maxlen)
    new_rb.extend(sorted(replay_buffer, key=lambda sample: sample[0]))
    return new_rb

(1 балл)

In [ ]:
def run_dqn_prioritized_rb(
        env_name="CartPole-v1",
        hidden_dims=(256, 256), lr=1e-3, gamma=0.99,
        eps_st=.4, eps_end=.02, eps_dur=.25, total_max_steps=100_000,
        train_schedule=4, replay_buffer_size=400, batch_size=32,
        eval_schedule=1000, smooth_ret_window=5, success_ret=200.
):
    env = gym.make(env_name)
    replay_buffer = deque(maxlen=replay_buffer_size)
    eval_return_history = deque(maxlen=smooth_ret_window)

    Q = create_network(
        input_dim=env.observation_space.shape[0], hidden_dims=hidden_dims,
        output_dim=env.action_space.n
    )
    opt = torch.optim.Adam(Q.parameters(), lr=lr)

    s, _ = env.reset()
    done = False

    for global_step in range(1, total_max_steps + 1):
        epsilon = linear(
            eps_st, eps_end, eps_dur * total_max_steps, global_step
        )
        a = select_action_eps_greedy(Q, s, epsilon=epsilon)
        s_next, r, terminated, truncated, _ = env.step(a)

        # Compute new sample loss (compute w/o gradients!)
        ####### Здесь ваш код ########
        with torch.no_grad():
            s_tensor = to_tensor([s])
            a_tensor = to_tensor([a], int).long()
            Q_s_all = Q(s_tensor)
            Q_s_a = Q_s_all[0, a_tensor[0]]
            
            td_target = compute_td_target(Q, [r], [s_next], [terminated], gamma=gamma)
            td_error = (Q_s_a - td_target[0]).abs()

            loss = td_error.item() + 1e-6 
        ##############################

        replay_buffer.append((loss, s, a, r, s_next, terminated))
        done = terminated or truncated

        if global_step % train_schedule == 0 and len(replay_buffer) >= batch_size:
            train_batch, indices = sample_prioritized_batch(
                replay_buffer, batch_size
            )
            (
                states, actions, rewards,
                next_states, terminateds
            ) = train_batch

            opt.zero_grad()
            td_target = compute_td_target(Q, rewards, next_states, terminateds, gamma=gamma)
            loss, td_losses = compute_td_loss(Q, states, actions, td_target, out_non_reduced_losses=True)
            loss.backward()
            opt.step()
            priorities = torch.sqrt(td_losses).detach().numpy()
            priorities = np.abs(priorities) + 1e-6
            update_batch(
                replay_buffer, indices, train_batch, priorities
            )

        # with much slower scheduler periodically re-sort replay buffer
        # such that it will overwrite the least important samples
        if global_step % (10 * train_schedule) == 0:
            replay_buffer = sort_replay_buffer(replay_buffer)

        if global_step % eval_schedule == 0:
            eval_return = eval_dqn(env_name, Q)
            eval_return_history.append(eval_return)
            avg_return = np.mean(eval_return_history)
            print(f'{global_step=} | {avg_return=:.3f} | {epsilon=:.3f}')
            if avg_return >= success_ret:
                print('Решено!')
                break

        s = s_next
        if done:
            s, _ = env.reset()
            done = False

run_dqn_prioritized_rb(eval_schedule=250)

global_step=250 | avg_return=25.000 | epsilon=0.396
global_step=500 | avg_return=17.500 | epsilon=0.392
global_step=750 | avg_return=20.000 | epsilon=0.389
global_step=1000 | avg_return=19.500 | epsilon=0.385
global_step=1250 | avg_return=27.000 | epsilon=0.381
global_step=1500 | avg_return=41.800 | epsilon=0.377
global_step=1750 | avg_return=49.400 | epsilon=0.373
global_step=2000 | avg_return=54.200 | epsilon=0.370
global_step=2250 | avg_return=62.400 | epsilon=0.366
global_step=2500 | avg_return=83.600 | epsilon=0.362
global_step=2750 | avg_return=78.200 | epsilon=0.358
global_step=3000 | avg_return=70.600 | epsilon=0.354
global_step=3250 | avg_return=77.400 | epsilon=0.351
global_step=3500 | avg_return=100.200 | epsilon=0.347
global_step=3750 | avg_return=95.000 | epsilon=0.343
global_step=4000 | avg_return=114.400 | epsilon=0.339
global_step=4250 | avg_return=131.800 | epsilon=0.335
global_step=4500 | avg_return=144.800 | epsilon=0.332
global_step=4750 | avg_return=148.800 | epsil